# Test for python script
Instead of SystemEngine data is mocked from JSON extracted from OC. 

In [1]:
import json
import math
# from flask import Flask, jsonify, request
fname = "Script Test.json"
with open(fname, "r") as json_file:
    f=json_file.read()
    jdata = json.loads(f)

def getElementByNameFromJson(el_name, jdata):
    for element in jdata["elements"]:
        if element["name"]==el_name:
            return element

In [2]:
#With this code we have generated Script Test.json so instead of systemEngine I will get the data from JSON

name1 = "plet1"
templateName1 ="OC-plet"
# position1 = systemEngine.vector(0, 0, -1500)
azimuth1 = 90
adjustable1 = True

name2 = "plet2"
templateName2 ="OC-plet"
# position2 = systemEngine.vector(-20000, 45000, -1500)
azimuth2 = 240
adjustable2 = True

# structure1 = systemEngine.addElementFromTemplate(name1, templateName1, position1, azimuth1, adjustable1)
structure1 = getElementByNameFromJson(name1,jdata)
# structure2 = systemEngine.addElementFromTemplate(name2, templateName2, position2, azimuth2, adjustable2)
structure2 = getElementByNameFromJson(name2,jdata)

lineName = "line1" 
lineTemplate = "CS line"
element1 = structure1
element2 = structure2
lineOd = 219.1
lineWallThickness = 4.5
lineColor = 255
lineRuleName = "rigid-pipe-rule"
# lineRuleParams = systemEngine.newRuleParameters().set("min-bend-radius", 200).set("min-straight-length", 500)
# line = systemEngine.addLine(lineName, lineTemplate, element1, element2, lineOd/1000, lineWallThickness, lineColor, lineRuleName, lineRuleParams)
# systemEngine.adjustElement(line.name, True, 100, 0.1)
line = getElementByNameFromJson(lineName,jdata)

In [3]:
# gettin min and max H for the wt calc
# H max is defined as Maximum Water Depth and in OC water depth is Z and negative
for prop in line["properties"]:
    if prop["name"]=="points":
        Hmax= -min([ point["z"] for point in prop["value"]])
        Hmin= -max([ point["z"] for point in prop["value"]])
print(Hmax, Hmin)

128.1952 -123.177574


In [4]:
def optimal_wt_DNV81(params):
    '''
    :param D: Outer Diameter [mm]
    :param t: Wall-Thickness [mm]
    :param Ca: Corrosion Allowance [mm]
    :param SMYS: Specified Maximum Yield Stress [MPa]
    :param E: Young's Modulus of the Material [MPa]
    :param Hmax: Maximum Water Depth [m]
    :param Pi: Internal Pipe Pressure [MPa]
    
    :param Nf: optional, default=0, Axial force (Positive for compressive) [N]
    :param SIGbA: Allowable Bending Stress (Percentage of SMYS) for Loading Condition a)
    :param SIGbB: Allowable Bending Stress (Percentage of SMYS) for Loading Condition b)
    :param LAT: Chart Datum [m]
    :param HAT: Highest Astronomical Tide [m]
    :param S: Storm Surge and Tide Height from 1 year Return Period [m]
    :param Wh: Maximum Wave Height [m]
    :param Pi: Internal Pipe Pressure [bar]
    :param MIxpA: Permissible Usage Factor for Critical Longitudinal Stress for Loading Condition a)
    :param MIypA: Permissible Usage Factor for Critical Longitudinal Stress for Loading Condition a)
    :param MIxpB: Permissible Usage Factor for Critical Hoop Stress for Loading Condition b)
    :param MIypB: Permissible Usage Factor for Critical Hoop Stress for Loading Condition b)
    :param g: Gravity constant [m/s^2]
    :param RHOw: Density of Water [kg/m^3]
    :param LC: Loading condition: "A" for functional loads,
                                  "B" for Design environmental load + Functional loads
                                  (according to 'DNV 1981 Appendix B')

    :return1 t_opt: Optimal Wall Thickness [mm]
    :return2 VAR: Variable for checking if Wall-Thickness is OK (if less than 1 -> Wall Thicknes is OK!)
    '''

    D = params["D"]/1000
    t = params["t"]/1000
    Ca = params["Ca"]/1000
    SMYS = params["SMYS"]*1e6
    E = params["E"]*1e6
    Hmax = params["Hmax"]
    Pi = params["Pi"]*1e5

    Nf = params["Nf"]
    SIGbA = params["SIGbA"]/100.0
    SIGbB = params["SIGbB"]/100.0
    LAT = params["LAT"]
    HAT = params["HAT"]
    S = params["S"]
    Wh = params["Wh"]
    g = params["g"]
    RHOw = params["RHOw"]
    LC = params["LC"]

    if LC == "A":
        MIxpA = 0.86
        MIypA = 0.75
    elif LC == "B":
        MIxpB = 1
        MIypB = 0.98
    else:
        print("Error, Loading condition must be 'A' or 'B'")


    def calculate(t):
        '''
        This function runs calculation according to DNV81.       
        :arg1 t: Wall-Thickness [mm]
        :return VAR: Variable for checking if Wall-Thickness is OK
        '''
        
        import math   
        dmax = Hmax+LAT+HAT+S+Wh/2
        Pe = RHOw*g*dmax
        Ac = math.pi*(D-t)*t
        Z = (math.pi/4)*(D-t)**2*t
        SIGnx = Nf/Ac
        SIGmxA = SIGbA*SMYS
        SIGxA = SIGnx+SIGmxA
        SIGmxB = SIGbB*SMYS
        SIGxB = SIGnx+SIGmxB
        var = D/t
        if var <= 20:
            SIGnxcr = SMYS
        elif var > 20:
            SIGnxcr = SMYS*(1-0.001*(var-20))
        elif var < 100:
            SIGnxcr = SMYS*(1-0.001*(var-20))
        else:
            print('Error: value D/t is too big')
        SIGmxcr = SMYS*(1.35-0.0045*var)
        SIGxcrA = (SIGnx/SIGxA)*SIGnxcr+(SIGmxA/SIGxA)*SIGmxcr
        SIGxcrB = (SIGnx/SIGxB)*SIGnxcr+(SIGmxB/SIGxB)*SIGmxcr
        SIGy = (abs(Pe-Pi)*(D/(2*t)))
        SIGye1 = E*(t/(D-t))**2
        SIGye2 = SMYS*(1-(1/3)*((2*SMYS)/(3*SIGye1))**2)
        if SIGye1 <= (2*SMYS)/3:
            SIGycr = E*(t/(D-t))**2
        elif SIGye2 > (2*SMYS)/3:
            SIGycr = SMYS*(1-(1/3)*((2*SMYS)/(3*SIGye1))**2)
        else:
            print('Error while calculating Critical Hoop Stress')
        ALFA = 1+(300/var)*(SIGy/SIGycr)

        if LC == "A":
            VAR = (SIGxA/(MIxpA*SIGxcrA))**ALFA+(SIGy/(MIypA*SIGycr))
        else:
            VAR = (SIGxB/(MIxpB*SIGxcrB))**ALFA+(SIGy/(MIypB*SIGycr))

        return VAR                                  #if less than 1 -> Wall Thicknes is OK!


    VAR = calculate(t)                              #calculate for inital parameter t
    flag = 0
    while flag < 1:
        if VAR < 1:                                 #if pass, the WT is maybe overdesigned
            t = t-0.0005                            #reduce the WT
            VAR = calculate(t)                      #run calculation for reduced WT
            if VAR >= 1:                            #if fails here, the previous WT is optimal
                t = t+0.0005                        #return WT to previous
                flag = 1                            #rise flag to 1 and exit 'while' loop
            else:
                pass
        elif VAR >= 1:                              #if not pass, WT is too low
            t = t+0.0005                            #increase the WT
            VAR = calculate(t)                      #run calculation for increased WT
            if VAR < 1:                             #if pass here, current WT is optimal
                flag = 1                            #rise flag to 1 and exit 'while' loop
            else:
                pass
        else:
            print('Error while calculating optimal Wall-Thickness')

    VAR = calculate(t)                              #run calculation one more time to get VAR
    t_opt = round(t*1000, 1)
    
    return t_opt, VAR



def minimal_wt_B314_B318(params):
    '''
    :param std: Standard according to ASME, must be "B31.4" or "B31.8"  *
    :param DC: Design case, must be "Hydrotest" (based on uncorroded pipe) or "Operating" (based on uncorroded pipe) *
    :param loc: Location, must be "Pipeline" or "Riser" or "Spool" *
    :param D: Outer Diameter [mm]
    :param t: Wall-Thickness [mm]
    :param Ca: Corrosion Allowance (for "Operating" case) [mm]
    :param Hmin: Minimum Water Depth [m]
    :param P: Design Pressure or Hydrotest Pressure [bar]
    :param T: Product temperature [Â°C]
    :param SMYS: Specified Maximum Yield Stress [MPa]
    :param RHOw: Density of Water [kg/m^3]
    :return1 t_min: Minimum Wall Thickness [mm]
    :return2 UR: Utilization Ratio (t_req/t_min)
    
    * for more info see Table 6.1 in "Pressure Containment Check in accordance with ASME B31.4 and B31.8"
    '''

    std = params["std"]
    DC = params["DC"]
    loc = params["loc"]
    D = params["D"]/1000
    t = params["t"]/1000
    Ca = params["Ca"]/1000
    Hmin = params["Hmin"]
    P = params["P"]*1e5
    T = params["T"]
    SMYS = params["SMYS"]*1e6
    RHOw = params["RHOw"]

    if (std != "B31.4") and (std != "B31.8"):
        print('Error, Standard must be "B31.4" or "B31.8"')
    if (DC != "Hydrotest") and (DC != "Operating"):
        print('Error, Design condition must be "Hydrotest" or "Operating"')
    if (loc != "Pipeline") and (loc != "Riser") and (loc != "Spool"):
        print('Error, Location must be "Pipeline" or "Riser" or "Spool"')

        
    def calculate(T):
        '''
        This function runs calculation according to ASME B31.4 / ASME B31.8
        
        :argument1 T: Product Temperature (Â°C)
        :return1 t_min: Minimum Required Pipeline Wall Thickness considering Corrosion Allowance [mm]
        :return2 UR: Utilization Ratio (nominal WT/t_min)
        '''
        
        if (std == "B31.4") and ((loc == "Riser") or (loc =="Spool")):
            F1 = 0.6
        elif (std == "B31.8") and ((loc == "Riser") or (loc =="Spool")):
            F1 = 0.5
        elif (DC == "Hydrotest"):
            F1 = 0.9
        else:
            F1 = 0.72

        g = 9.807
        Pmin = RHOw*g*Hmin
        if T <= 121:
            Tder = 1
        elif (T > 121) and (T <= 148.9):
            Tder = ((T-121)/(148.9-121))*(0.967-1)+1
        elif (T > 148.9) and (T <= 176.7):
            Tder = ((T-148.9)/(176.7-148.9))*(0.933-0.967)+0.967
        elif (T > 176.7) and (T <= 204.4):
            Tder = ((T-176.7)/(204.4-176.7))*(0.9-0.933)+0.933
        elif (T > 204.4) and (T <= 232.2):
            Tder = ((T-204.4)/(232.2-204.4))*(0.867-0.9)+0.9
        else:
            print("Error, Temperature of product can not be higher than 232.2 Â°C")
 
        tmin = ((P-Pmin)*D)/(2*F1*SMYS*Tder)
        tmin = tmin *1000
        t_req = tmin + Ca

        return t_req


    t_req = calculate(t)
    t_min = round(t_req*2+0.5)/2            #round Wall Thickness to the nearest upper 0.5 of number
    UR = t_req/t_min
    return t_min, UR



def routeCurveRadius(params):
    '''
    :param D: Outer Diameter [mm]
    :param t: Wall-Thickness [mm]
    :param t_ca: Corrosion Allowance [mm]
    :param t_clad: Internal Clad Thickness [mm]
    :param RHOst: Steel Pipe Density [kg/m^3]
    :param RHOclad: Internal Clad Density [kg/m^3]
    :param E: Modulus of Elasticity [MPa]
    :param SMYS: Pipeline Specified Minimum Yield Strength [MPa]
    :param t_ac: Anti-Corrosion Coating Thickness [mm]
    :param RHOac: Anti-Corrosion Coating Density [kg/m^3]
    :param t_cc: Concrete Coating Thickness [mm]
    :param RHOcc: Concrete Coating Density [kg/m^3]
    :param L_fj_coat: Field Joint Coating Length for Anti-Corrosion Coating Cutback at Each Pipe End [mm]
    :param RHOfj_coat: Field Joint Coating Density for Anti-Corrosion Coating Cutback [kg/m^3]
    :param L_fj_conc: Field Joint Coating Length for Concrete Coating Cutback at Each Pipe End [mm]
    :param RHOfj_conc: Field Joint Coating Density for Concrete Coating Cutback [kg/m^3]
    :param L_pj: Pipe Joint length [m]
    :param RHOw: Seawater Density [kg/m^3]
    :param MIlat: Lateral Soil Friction Coefficient 
    :param MIlong: Longitudinal Soil Friction Coefficient
    :param SIGMAs: Seabed Slope Angle [deg]
    :param Ttd: Maximum On-bottom Lay Tension at touch down during Laying [kN]
    :param FOSmin_cr: Minimum Factor of Safety for Curve Pulling due to On-bottom Lay Tension
    :param SIGb: Permitted Bending Stress Percentage due to Lay Radius [%]
    :param R_proposed: Proposed Pipeline Route Curve Radius [m]

    :return1 R_min: Minimum Pipeline Route Curve Radius   [m]
    :return2 L_min: Minimum Required Straight Length on Seabed prior to Curve [m]
    '''
    
    import math
    D = params["D"]/1000.0
    t = params["t"]/1000.0
    t_ca = params["t_ca"]/1000.0
    t_clad = params["t_clad"]/1000.0
    RHOst = params["RHOst"]
    RHOclad = params["RHOclad"]
    SMYS = params["SMYS"]*1e6
    E = params["E"]*1e6
    t_ac = params["t_ac"]/1000.0
    RHOac = params["RHOac"]
    t_cc = params["t_cc"]/1000.0
    RHOcc = params["RHOcc"]
    L_fj_coat = params["L_fj_coat"]/1000.0
    RHOfj_coat = params["RHOfj_coat"]
    L_fj_conc = params["L_fj_conc"]/1000.0
    RHOfj_conc = params["RHOfj_conc"]
    L_pj = params["L_pj"]
    RHOw = params["RHOw"]
    MIlat = params["MIlat"]
    MIlong = params["MIlong"]
    SIGMAs = math.radians(params["SIGMAs"])
    Ttd = params["Ttd"]*1000.0
    FOSmin_cr = params["FOSmin_cr"]
    SIGb = params["SIGb"]/100.0
    R_proposed = params["R_proposed"]
    g = 9.807


    Do = D+2*t_ac+2*t_cc
    As = math.pi*(t-t_ca)*(D-(t-t_ca))
    Aclad = (math.pi/4)*((D-2*t)**2-(D-2*t-2*t_clad)**2)
    Aac = (math.pi/4)*((D+2*t_ac)**2-D**2)
    Acc = (math.pi/4)*((D+2*t_ac+2*t_cc)**2-(D+2*t_ac)**2)
    Ms = As*RHOst*L_pj
    Mclad = Aclad*RHOclad*L_pj
    Mac = Aac*RHOac*(L_pj-2*L_fj_coat)
    Mcc = Acc*RHOcc*(L_pj-2*L_fj_conc)
    Mfj_coat = Aac*RHOfj_coat*2*L_fj_coat
    Mfj_conc =  Acc*RHOfj_conc*2*L_fj_conc
    Mc = 0
    B = (math.pi/4)*Do**2*RHOw*g*L_pj
    Wa = (Ms+Mclad+Mac+Mcc+Mfj_coat+Mfj_conc+Mc)*g
    Ws_pj = Wa-B
    Ws = Ws_pj/L_pj

    Rcp = (Ttd)/(Ws*(MIlat*math.cos(SIGMAs)-math.sin(SIGMAs)))
    Rbs = (0.5*D*E)/(SIGb*SMYS)
    R = max(Rcp, Rbs)
    R_min = R*FOSmin_cr
    R_min = round((R_min+50)/100)*100           #rounds number to upper 100. Does this make sense to do?
    
    L_min = Ttd/(MIlong*Ws)
    L_min = round((L_min+50)/100)*100           #rounds number to upper 100. Does this make sense to do?
    return R_min, L_min



#Hmax je z koordinata od tocaka linije
params_DNV81 = {"D" : 219.1, "t" : 4.5, "Ca" : 0, "SMYS" : 450, "E" : 2.07E5, "Hmax" : 65.27, "Pi" : 0,
    "Nf" : 0, "SIGbA" : 85, "SIGbB" : 85, "LAT" : 0, "HAT" : 1.06, "S" : 0, "Wh" : 6.3,
    "g" : 9.807, "RHOw" : 1025, "LC" : "A"
    }
DNV81 = optimal_wt_DNV81(params_DNV81)


params_B314_B318 ={"std" : "B31.4", "DC" : "Operating", "loc" : "Spool", "D" : 323.9, "t" : 9.5,
    "Ca" : 3, "Hmin" : 52.5, "P" : 92.5, "T" : 125, "SMYS" : 448, "RHOw" : 1025
    }
B314_B318 = minimal_wt_B314_B318(params_B314_B318)

params_curveRadius = {"D" : 273.1, "t" : 18.3, "t_ca" : 0, "t_clad" : 3, "RHOst" : 7850, "RHOclad" : 8027,
   "E" : 207000, "SMYS" : 450, "t_ac" : 3, "RHOac" : 951, "t_cc" : 30, "RHOcc" : 3044, "L_fj_coat" : 300,
   "RHOfj_coat" : 900, "L_fj_conc" : 300, "RHOfj_conc" : 900, "L_pj" : 12, "RHOw" : 1025, "MIlat" : 0.4,
   "MIlong" : 0.2, "SIGMAs" : 1, "Ttd" : 300, "FOSmin_cr" : 1.5, "SIGb" : 10, "R_proposed" : 1600
    }
curveRadius = routeCurveRadius(params_curveRadius)


In [5]:
DNV81

(4.5, 0.9975132330787537)

In [6]:
B314_B318

(5.5, 0.9560176803195176)

In [7]:
curveRadius

(1000, 1200)

In [8]:
!git pull

Username for 'https://github.com': ^C
